In [2]:
%matplotlib inline

import sys
sys.path.append("../")

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from IPython.display import Markdown, display

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


#### Load dataset and set options

In [3]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

#### Metric for original training data

In [4]:
# Metric for the original dataset
privileged_groups = [{'sex': 1.0}]
unprivileged_groups = [{'sex': 0.0}]

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.193662


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.196507


#### Train with and transform the original training data

In [5]:
scale_orig = StandardScaler()
dataset_orig_train.features = scale_orig.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = scale_orig.transform(dataset_orig_test.features)

In [6]:
# Input recontruction quality - Ax
# Fairness constraint - Az
# Output prediction error - Ay

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
    
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
TR = TR.fit(dataset_orig_train, maxiter=5000, maxfun=5000)

step: 0, loss: 0.8248745006364724, L_x: 2.4562291653747206,  L_y: 0.556274147762223,  L_z: 0.01148871816838871
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          190     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.24875D-01    |proj g|=  1.70678D-02
step: 250, loss: 0.8225663928658248, L_x: 2.4542585887813217,  L_y: 0.5551546650160966,  L_z: 0.01099293448579802

At iterate    1    f=  8.22566D-01    |proj g|=  1.82058D-02
step: 500, loss: 0.8010326580481005, L_x: 2.381059755824986,  L_y: 0.5402945693956658,  L_z: 0.01131605653496808

At iterate    2    f=  8.01033D-01    |proj g|=  1.95040D-02
step: 750, loss: 0.7842317548189357, L_x: 2.3602522698335915,  L_y: 0.5332522979837272,  L_z: 0.007477114925924687

At iterate    3    f=  7.84232D-01    |proj g|=  1.80112D-02

At iterate    4    f=  7.47652D-01    |proj g|=  2.33976D-02
step: 1000, loss: 0.7149587539517059, L_x: 2.0770765868493277,  L_y: 0

In [7]:
# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_train)
dataset_transf_test = TR.transform(dataset_orig_test)

In [8]:
print(classification_report(dataset_orig_test.labels, dataset_transf_test.labels))

              precision    recall  f1-score   support

         0.0       0.83      0.91      0.87     11133
         1.0       0.59      0.41      0.49      3520

    accuracy                           0.79     14653
   macro avg       0.71      0.66      0.68     14653
weighted avg       0.77      0.79      0.78     14653


In [13]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Training dataset"))
print("Transformed: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())
print("Original: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Test dataset"))
print("Transformed: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_test.mean_difference())
print("Original: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Training dataset

Transformed: Difference in mean outcomes between unprivileged and privileged groups = -0.072447
Original: Difference in mean outcomes between unprivileged and privileged groups = -0.193662


#### Test dataset

Transformed: Difference in mean outcomes between unprivileged and privileged groups = -0.079498
Original: Difference in mean outcomes between unprivileged and privileged groups = -0.196507


abs(1-disparate impact) must be small (close to 0) for classifier predictions to be fair.

In [10]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_transf_train.consistency())
print("Consistency of labels in original training dataset= %f" %metric_orig_train.consistency())
print("Consistency of labels in transformed test dataset= %f" %metric_transf_test.consistency())
print("Consistency of labels in original test dataset= %f" %metric_orig_test.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 1.000000
Consistency of labels in original training dataset= 0.754225
Consistency of labels in transformed test dataset= 1.000000
Consistency of labels in original test dataset= 0.721545


In [12]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_transf_test.consistency() > metric_orig_test.consistency(),\
        "Transformed dataset consistency should be greater than original dataset."
    
    print('Success!')

check_algorithm_success()    

Success!
